In [2]:
# Generate message concat values

In [2]:
import asyncio
import requests
import time
import numpy as np
import nest_asyncio 

import pandas as pd
from influxdb import DataFrameClient

In [3]:

nest_asyncio.apply()

In [4]:
now = time.time() # 'UTC'
now

1603605343.0150635

In [5]:
pd.Timestamp.fromtimestamp(now) # 'UTC' Coordinated Universal Time

Timestamp('2020-10-25 05:55:43.015064')

In [6]:
# kl1124 KL571


#if using datetime.datetime.now().tz_localize(tz='Europe/Copenhagen')

pd.Timestamp.fromtimestamp(now).tz_localize(tz='UTC').tz_convert(tz='Europe/Copenhagen') 

Timestamp('2020-10-25 06:55:43.015064+0100', tz='Europe/Copenhagen')

In [7]:
URI = "https://data-live.flightradar24.com/zones/fcgi/feed.js"
params = dict(
    bounds= [56.75,55.20,4.59,17.27],
    faa= 1,
    satellite= 1,
    mlat= 1,
    flarm= 1,
    adsb= 1,
    gnd= 1,
    air= 1,
    vehicles= 1,
    estimated= 1,
    maxage= 14400,
    gliders= 1,
    stats= 1,
)

In [8]:
session = requests.Session()

In [9]:
session.headers.update({'User-Agent':'Pandas'})

In [10]:
columns = {
    0: 'uuid',
    1: 'icao_id',
    2: 'latitude',
    3: 'longitude',
    4: 'track_degree',
    5: 'calibrated_altitude',
    6: 'ground_speed_kts',
    7: 7,
    8: 'receiver_code',
    9: 'aircraft_type',
    10: 'registration',
    11: 'update_time_utc',
    12: 'from',
    13: 'to',
    14: 'flight_number',
    15: 15,
    16: 16,
    17: 'flight_tag',
    18: 18,
    19: 'airline',
}

In [11]:
fields = ['uuid',
'icao_id',
'latitude',
'longitude',
'track_degree',
'ground_speed_kts',
'from',
'to',
'flight_number',
'flight_tag',
'airline']

In [12]:
r = session.get(URI, params=params)

now = time.time()
call_time = pd.Timestamp.fromtimestamp(
        now).tz_localize(tz='UTC'
           ).tz_convert(tz='Europe/Copenhagen')
    
data =r.json()

_ = [value.insert(0,key) for key, value in data.items() if isinstance(value, list)]

results = [value for key, value in data.items() if isinstance(value, list)]

df = pd.DataFrame(results).rename(columns, axis=1)
df = df.replace({'': np.nan})
df['call_dt'] = call_time
df.set_index('call_dt', drop=True, inplace=True)
df.dropna(inplace=True)

df = df[fields]
df.shape

(1100, 11)

Fields are data
Tags are metadata. 
Fields can either be indexed or not indexed. 
Fields that are indexed are called tags.
Tag if field is going to be indexed

In [13]:
df.columns

Index(['uuid', 'icao_id', 'latitude', 'longitude', 'track_degree',
       'ground_speed_kts', 'from', 'to', 'flight_number', 'flight_tag',
       'airline'],
      dtype='object')

In [14]:
user = 'danpra'
password = 'influxdbpwd'
host = 'influxdb'
port = 8086
dbname= 'flights'
protocol = 'line'

client = DataFrameClient(host, port, user, password, dbname)
client.create_database(dbname)

In [15]:
client.get_list_measurements()

[]

In [16]:
{client.drop_measurement(item['name']) for item in client.get_list_measurements()}

set()

In [17]:
data['full_count']

5761

In [18]:
tags =['from', 'to', 'airline']

async def get_data(params=params):
    
    now = time.time()
    call_time = pd.Timestamp.fromtimestamp(
        now).tz_localize(tz='UTC'
           ).tz_convert(tz='Europe/Copenhagen')
    
    
    r = session.get(URI, params=params)

    data =r.json()

    _ = [value.insert(0,key) for key, value in data.items() if isinstance(value, list)]

    results = [value for key, value in data.items() if isinstance(value, list)]
    
    df = pd.DataFrame(results).rename(columns, axis=1)
    df = df.replace({'': np.nan})
    df['call_dt'] = call_time
    df.set_index('call_dt', drop=True, inplace=True)
    
    df.dropna(inplace=True)
    df = df[fields]
    
    client.write_points(df, 'airlines', tag_columns=tags, protocol='line')
    print(f'Time: {call_time} Data: {df.shape[0]} airlines spotted')
    

In [19]:
async def forever(snooze=2):
    while True:
        await get_data()
        await asyncio.sleep(snooze)

In [20]:
loop = asyncio.get_event_loop()

In [ ]:
loop.run_until_complete(forever(1))

Time: 2020-10-25 06:56:00.206398+01:00 Data: 1099 airlines spotted
Time: 2020-10-25 06:56:01.495960+01:00 Data: 1099 airlines spotted
Time: 2020-10-25 06:56:02.708557+01:00 Data: 1100 airlines spotted
Time: 2020-10-25 06:56:03.884158+01:00 Data: 1099 airlines spotted
Time: 2020-10-25 06:56:05.090661+01:00 Data: 1099 airlines spotted
Time: 2020-10-25 06:56:06.292116+01:00 Data: 1098 airlines spotted
Time: 2020-10-25 06:56:07.488721+01:00 Data: 1098 airlines spotted
Time: 2020-10-25 06:56:08.696294+01:00 Data: 1100 airlines spotted
Time: 2020-10-25 06:56:09.903001+01:00 Data: 1100 airlines spotted
Time: 2020-10-25 06:56:11.104319+01:00 Data: 1101 airlines spotted
Time: 2020-10-25 06:56:12.503490+01:00 Data: 1098 airlines spotted
Time: 2020-10-25 06:56:13.702652+01:00 Data: 1099 airlines spotted
Time: 2020-10-25 06:56:14.899030+01:00 Data: 1099 airlines spotted
Time: 2020-10-25 06:56:16.099385+01:00 Data: 1097 airlines spotted
Time: 2020-10-25 06:56:17.303453+01:00 Data: 1097 airlines spo

In [34]:
loop.stop()